In [1]:
import os
import pandas as pd

In [14]:
train_path = 'Literature Data/train_set_3_0'
words_path = train_path
csv_name = 'Literature Data/train_3_0.csv'
names = []
authors = []
for word_file in os.listdir(words_path):
    label =  word_file[:word_file.find('word' )-1]
    names.append(word_file)
    authors.append(label)
pd.DataFrame({"file_name": names, "label": authors}) \
        .to_csv(csv_name, index=False, header=True, columns = ["file_name", "label"])  

In [15]:
train_path = 'Literature Data/test_set_3_0'
words_path = train_path
csv_name = 'Literature Data/test_3_0.csv'
names = []
authors = []
for word_file in os.listdir(words_path):
    label =  word_file[:word_file.find('word' )-1]
    names.append(word_file)
    authors.append(label)
pd.DataFrame({"file_name": names, "label": authors}) \
        .to_csv(csv_name, index=False, header=True, columns = ["file_name", "label"])  

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import cv2
from os.path import join


def fetch(img_dir, name):
    img = cv2.imread(join(img_dir, name))
    if img.shape == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


def resize(img, size=(1024, 768)):
    assert len(size) == 2
    return cv2.resize(img, size, interpolation=cv2.INTER_CUBIC)


def pad(img, size=(1024, 768)):
    assert len(img.shape) == 3
    assert len(size) == 2
    h, w, _ = img.shape
    #assert w <= size[0] and h <= size[1]
    pad_vert = np.ceil((size[1]-h) / 2).astype(np.uint32)
    pad_hor = np.ceil((size[0]-w) / 2).astype(np.uint32)

    padded = np.full((size[1], size[0], 3), 255).astype(np.uint8)
    padded[pad_vert:pad_vert+h, pad_hor:pad_hor+w, :] = img.copy()
    return padded



In [5]:
import sys 
sys.path.insert(0, '../')
import numpy as np
import pandas as pd
from keras.utils import Sequence
from sklearn.utils import shuffle

from keras.utils import to_categorical

    
class WordsSequence(Sequence):
    def __init__(self, img_dir, input_shape, x_set, y_set=None, batch_size=1, classification=False):
        if classification:
            if y_set is not None:
                self.x, self.y = x_set, y_set
                self.dataset = pd.DataFrame(data={'x': self.x, 'y': self.y, 'used': np.zeros_like(self.y)})
            else:
                self.x, self.y = x_set, None
        else:
            if y_set is not None:
                self.x, self.y = x_set, y_set
                self.x, self.y = shuffle(self.x, self.y)
            else:
                self.x, self.y = x_set, None

        self.img_dir = img_dir
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.classification = classification


    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        if self.classification:
            if self.y is None:
                batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
                return np.array([self.preprocess(fetch(self.img_dir, name)) for name in batch_x])

            unused = self.dataset.loc[self.dataset['used'] == 0]
            if len(unused) >= self.batch_size:
                batch_indices = unused.sample(n=self.batch_size).index
            else:
                batch_indices = unused.sample(n=self.batch_size, replace=True).index

            self.dataset.loc[batch_indices, 'used'] = 1
            batch_x = self.dataset.iloc[batch_indices]['x'].values
            batch_y = self.dataset.iloc[batch_indices]['y'].values 
            ##########################
            # Здесь вместо 108 - количество классов
            return np.array([self.preprocess(fetch(self.img_dir, name)) for name in batch_x]), to_categorical(batch_y, 94)

        if self.y is None:
            x = self.x[idx]
            return np.expand_dims(self.preprocess(fetch(self.img_dir, x)), axis=0)


        curr_x = self.x[idx]
        curr_y = self.y[idx]

        x_1_images = self.preprocess(fetch(self.img_dir, curr_x[0]))
        x_2_images = self.preprocess(fetch(self.img_dir, curr_x[1]))
        return [np.expand_dims(x_1_images, axis=0), np.expand_dims(x_2_images, axis=0)], np.array([curr_y])


       

    def preprocess(self, img):
        assert len(img.shape) == 3

        h, w, _ = img.shape
        if h / w <= self.input_shape[0] / self.input_shape[1]:
            img = resize(img, (self.input_shape[1], int(self.input_shape[1] * h / w)))
        else:
            img = resize(img, (int(self.input_shape[0] * w / h), self.input_shape[0]))

        img = pad(img, (self.input_shape[1], self.input_shape[0]))
        return img / 255. 

    def on_epoch_end(self):
        if (not self.classification) and (self.y is not None):
            self.x, self.y = shuffle(self.x, self.y)
        
        if self.classification and self.y is not None:
            self.dataset = pd.DataFrame(data={'x': self.x, 'y': self.y, 'used': np.zeros_like(self.y)})
            self.dataset = self.dataset.sample(n=len(self.dataset))


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
import os
import numpy as np
import pandas as pd
from collections import Counter



from keras import backend as K
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.applications.mobilenet import MobileNet
from keras.layers import Input, Dense, Lambda, Dropout, Activation
from keras.optimizers import rmsprop, Adam, SGD, Adagrad


from keras.utils import to_categorical
from keras.models import Model


def get_str2numb_numb2dict(vect):
    str_to_ind_dict = {}
    count = 0
    for v in vect:
        if v not in str_to_ind_dict.keys():
            str_to_ind_dict[v] = count
            count += 1
    reverse_dict = {v:k for k, v in str_to_ind_dict.items()}
    return str_to_ind_dict, reverse_dict

    
def apply_dict(dict_keys, X):
    res = []
    for x in X:
        if x in dict_keys.keys():
            res.append(dict_keys[x])
        else:
            res.append(-1)
    return res
    
    
class Classification_model:
    def __init__(self, alpha, input_shape, num_classes, cache_dir, train_head, batch_size=32):
        self.alpha = alpha
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.cache_dir = cache_dir
        self.batch_size = batch_size
        if not os.path.isdir(self.cache_dir):
            os.makedirs(self.cache_dir)
        self.build_model(train_head)
    
    def build_model(self, train_head):        
        # If imagenet weights are being loaded, alpha can be one of`0.25`, `0.50`, `0.75` or `1.0` only.
        base_model = MobileNet(input_shape=self.input_shape, alpha=self.alpha, weights='imagenet', 
                               include_top=False, pooling='avg') 
        
        # Use to pretrain head
        # if train_head:
            # for layer in base_model.layers[:-4]:
                # layer.trainable = False
            
        # base_model.summary()
        op = Dense(128, activation='relu')(base_model.output)
        op = Dropout(0.00001)(op)
        output_tensor = Dense(self.num_classes, activation='softmax')(op)
       
        self.model = Model(inputs=base_model.input, outputs=output_tensor)
        self.model.summary()
        
    def train(self, train_dir, train_csv, epochs, learning_rate=0.00001):
        train = pd.read_csv(train_csv)
        train_x, train_y = train['file_name'].as_matrix(), train['label'].as_matrix()
        
        self.str2ind_dict, self.ind2str_dict = get_str2numb_numb2dict(train_y)
        train_y = np.array(apply_dict(self.str2ind_dict, train_y))

        train_generator = WordsSequence(img_dir=train_dir,
                                        input_shape = self.input_shape,
                                        x_set=train_x,
                                        y_set=train_y,
                                        batch_size=self.batch_size,
                                        classification=True)
                                        
        optimize = rmsprop(lr=learning_rate, decay=1e-6)
        # optimize = Adam(lr=0.00000001) 
        # optimize = SGD()
        # optimize = Adagrad(lr=0.0001)
        
        self.model.compile(loss='categorical_crossentropy', optimizer=optimize, metrics=['categorical_accuracy'])
        
        self.model.fit_generator(train_generator,
                                 steps_per_epoch=len(train_x)//self.batch_size,
                                 shuffle=True,
                                 epochs=epochs,
                                 verbose=1, 
                                 callbacks=[ModelCheckpoint(filepath=file_path_to_checkpoint, save_weights_only=True)])
        path_to_save_model = self.cache_dir + '/' + 'final_model.h5'
        path_to_save_weights = self.cache_dir + '/' + 'final_weights.h5'
        
    def save_weights(self, filename):
        self.model.save_weights(filename)
        
    def load_weights(self, filename):
        self.model.load_weights(filename, by_name=True, skip_mismatch=True)
        
    def predict(self, test_dir, test_csv): 

        test = pd.read_csv(test_csv)
        test_x, test_y = test['file_name'].as_matrix(), test['label'].as_matrix()
        self.str2ind_dict, self.ind2str_dict = get_str2numb_numb2dict(test_y)
        test_generator = WordsSequence(img_dir=test_dir,
                                        input_shape = self.input_shape,
                                        x_set=test_x,
                                        batch_size=self.batch_size,
                                        classification=True)
                                        
        pred = np.argmax(self.model.predict_generator(test_generator, verbose=1), axis=1)  
        res = np.array(apply_dict(self.ind2str_dict, pred))
        
        count = 0
        for i,j in zip(res, test_y):
            if i == j:
                count += 1
        print('word accuracy: ', count / len(test_y))
        
        count = 0
        autors = np.unique(test_y)
        autor_ind = [np.argwhere(test_y == a) for a in autors]
        for i,inds in enumerate(autor_ind):
            p = Counter(np.ravel(res[inds])).most_common(1)[0][0]
            if p == autors[i]:
                count += 1

        print('top-1 autor accuracy: ', count / len(autors))
        
        сount = 0
        for i,inds in enumerate(autor_ind):
            p = [pair[0] for pair in Counter(np.ravel(res[inds])).most_common(5)]
            if autors[i] in p:
                сount += 1

        print('top-5 autor accuracy: ', сount / len(autors))


In [7]:
size=(1024, 768)
count = 0
test_dir = 'Literature data/test_set_3_0'
imgs = os.listdir(test_dir)
for j, img_p in enumerate(imgs):
    path = test_dir + '/' + img_p
    img = cv2.imread(path)
    assert len(img.shape) == 3
    assert len(size) == 2
    h, w, _ = img.shape
    if w > size[0] or h > size[1]:
        os.remove(path)
        count += 1
print(count)


27


In [8]:
size=(1024, 768)
count = 0
test_dir = 'Literature data/train_set_3_0'
imgs = os.listdir(test_dir)
for j, img_p in enumerate(imgs):
    path = test_dir + '/' + img_p
    img = cv2.imread(path)
    assert len(img.shape) == 3
    assert len(size) == 2
    h, w, _ = img.shape
    if w > size[0] or h > size[1]:
        os.remove(path)
        count += 1
print(count)


35


In [16]:
cache_dir = 'Literature data/cache_3_0'
train_dir = 'Literature data/train_set_3_0'
test_dir = 'Literature data/test_set_3_0'
# Train
model = Classification_model(alpha=1, input_shape=(160,160,3), num_classes=94, cache_dir=cache_dir, train_head=False)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160, 160, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 161, 161, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 80, 80, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 80, 80, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 80, 80, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 80, 80, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 80, 80, 32)        128 

In [19]:
self = model
learning_rate=0.00001
train_csv = 'Literature Data/train_3_0.csv'
test_csv = 'Literature Data/test_3_0.csv'
train = pd.read_csv(train_csv)
train_x, train_y = train['file_name'].as_matrix(), train['label'].as_matrix()

self.str2ind_dict, self.ind2str_dict = get_str2numb_numb2dict(train_y)
train_y = np.array(apply_dict(self.str2ind_dict, train_y))

train_generator = WordsSequence(img_dir=train_dir,
                                input_shape = self.input_shape,
                                x_set=train_x,
                                y_set=train_y,
                                batch_size=self.batch_size,
                                classification=True)

optimize = rmsprop(lr=learning_rate, decay=1e-6)
# optimize = Adam(lr=0.00000001) 
# optimize = SGD()
# optimize = Adagrad(lr=0.0001)

self.model.compile(loss='categorical_crossentropy', optimizer=optimize, metrics=['categorical_accuracy'])


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [20]:
epochs = 100
file_path_to_checkpoint=self.cache_dir + '/checkpoint-{epoch:02d}.h5'
self.model.fit_generator(train_generator,
                                 steps_per_epoch=len(train_x)//self.batch_size,
                                 shuffle=True,
                                 epochs=epochs,
                                 verbose=1, 
                                 callbacks=[ModelCheckpoint(filepath=file_path_to_checkpoint, save_weights_only=True)])

Epoch 1/100
100/100 [==============================] - 701s 7s/step - loss: 4.7955 - categorical_accuracy: 0.0075
Epoch 2/100
100/100 [==============================] - 703s 7s/step - loss: 4.5018 - categorical_accuracy: 0.0272
Epoch 3/100
100/100 [==============================] - 708s 7s/step - loss: 4.2956 - categorical_accuracy: 0.0541
Epoch 4/100
100/100 [==============================] - 713s 7s/step - loss: 4.1343 - categorical_accuracy: 0.1000
Epoch 5/100
100/100 [==============================] - 709s 7s/step - loss: 3.9690 - categorical_accuracy: 0.1391
Epoch 6/100
100/100 [==============================] - 680s 7s/step - loss: 3.8145 - categorical_accuracy: 0.1641
Epoch 7/100
100/100 [==============================] - 680s 7s/step - loss: 3.6820 - categorical_accuracy: 0.1947
Epoch 8/100
100/100 [==============================] - 682s 7s/step - loss: 3.5440 - categorical_accuracy: 0.2353
Epoch 9/100
100/100 [==============================] - 687s 7s/step - loss: 3.4009 - cat

100/100 [==============================] - 664s 7s/step - loss: 0.0757 - categorical_accuracy: 0.9866
Epoch 73/100
100/100 [==============================] - 665s 7s/step - loss: 0.0794 - categorical_accuracy: 0.9822
Epoch 74/100
100/100 [==============================] - 654s 7s/step - loss: 0.0685 - categorical_accuracy: 0.9856
Epoch 75/100
100/100 [==============================] - 647s 6s/step - loss: 0.0771 - categorical_accuracy: 0.9781
Epoch 76/100
100/100 [==============================] - 656s 7s/step - loss: 0.0749 - categorical_accuracy: 0.9809
Epoch 77/100
100/100 [==============================] - 685s 7s/step - loss: 0.0689 - categorical_accuracy: 0.9819
Epoch 78/100
100/100 [==============================] - 650s 7s/step - loss: 0.0711 - categorical_accuracy: 0.9812
Epoch 79/100
100/100 [==============================] - 646s 6s/step - loss: 0.0637 - categorical_accuracy: 0.9822
Epoch 80/100
100/100 [==============================] - 651s 7s/step - loss: 0.0621 - categor

In [21]:
path_model = 'Literature Data/cache_3_0/my_model.h5'
self.model.save(path_model)
path_to_save_weights = self.cache_dir + '/' + 'final_weights.h5'
self.save_weights(path_to_save_weights)

In [22]:
test = pd.read_csv(test_csv)
test_x, test_y = test['file_name'].as_matrix(), test['label'].as_matrix()
self.str2ind_dict, self.ind2str_dict = get_str2numb_numb2dict(test_y)
test_generator = WordsSequence(img_dir=test_dir,
                                input_shape = self.input_shape,
                                x_set=test_x,
                                batch_size=self.batch_size,
                                classification=True)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [23]:
pred = np.argmax(self.model.predict_generator(test_generator, verbose=1), axis=1)  
res = np.array(apply_dict(self.ind2str_dict, pred))

79/79 [==============================] - 219s 3s/step


In [24]:
count = 0
for i,j in zip(res, test_y):
    if i == j:
        count += 1
print('word accuracy: ', count / len(test_y))

count = 0
autors = np.unique(test_y)
autor_ind = [np.argwhere(test_y == a) for a in autors]
for i,inds in enumerate(autor_ind):
    p = Counter(np.ravel(res[inds])).most_common(1)[0][0]
    if p == autors[i]:
        count += 1

print('top-1 autor accuracy: ', count / len(autors))

сount = 0
for i,inds in enumerate(autor_ind):
    p = [pair[0] for pair in Counter(np.ravel(res[inds])).most_common(5)]
    if autors[i] in p:
        сount += 1

print('top-5 autor accuracy: ', сount / len(autors))


word accuracy:  0.16939019529693106
top-1 autor accuracy:  0.30851063829787234
top-5 autor accuracy:  0.5957446808510638
